まずはデータの読み込みをした後、すべての出現単語を記録した辞書dicを作成する(辞書といいつつデータフレーム型ですすみません)。  
今後必要になるのは
- すべてのセリフを格納したserif_all
- すべての名前を格納したname_all
- すべてのタイプを格納したtype_all
- 単語を出現頻度順に並べた辞書のようなものdic(今後は単語をこの"辞書"のidに変換して扱う)

である、どのような感じかだけつかんでおけばよい。

In [1]:
import codecs
import numpy as np
import pandas
import random
import sys
from janome.tokenizer import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding

#セリフのデータを読みこむ
types = ["cute","cool","passion"]
serif_all = []
name_all = []
type_all = []#0はcute,1はcool,2はpassion
for i,type_temp in enumerate(types):#タイプごとにデータを読み込む
    filename = "./data/comment_"+type_temp+".txt"#アイドルコメントの読み込み
    file = codecs.open(filename, 'r')
    serif_all.extend(file.readlines())
    type_all.extend([i]*(len(serif_all)-len(type_all)))#serif_allの長さ-type_allの長さが注目しているタイプのセリフ数
    file.close()
    filename = "./data/name_"+type_temp+".txt"#アイドル名の読み込み
    file = codecs.open(filename,"r")
    name_all.extend(file.readlines())
    file.close()

#品詞ごとに分解する
words = [] # 単語文字列、品詞、登場回数のリスト
t = Tokenizer()
counter = 0
for i in range(len(name_all)):#まずは下準備："\n"を削除
    name_all[i] = name_all[i].replace("\n","")
    serif_all[i] = serif_all[i].replace("\n","")
    
for serif in serif_all:
    counter += 1
    print(counter,end = "\r")
    tokens = t.tokenize(serif)#品詞ごとに分解される
    for j in range(0,len(tokens)): #すべての単語についてみていく
        token = tokens[j]#ある単語に注目、jを使わないとうまくいかなかったはず
        is_new_word = True
        for i in range(len(words)):#既出単語と比較し、tokenが既出か確認
            if words[i][0] == token.surface and words[i][1] == token.part_of_speech[:2]:#tokenの単語そのものと品詞が一致していたら既出とみなす
                words[i][2] += 1#出現回数を増やす
                is_new_word = False#既出だった場合の処理
                break
        if is_new_word:#新出単語であれば
            words.append([token.surface, token.part_of_speech[:2], 1])#wordsに記録、part_of_speech[:2]になっているのはpart_of_speechが"名詞,一般,*,*"のような文字列になっているから"
# 単語情報をデータフレームに変換
dic = pandas.DataFrame(words)
dic.columns = ['words', 'parts', 'freq']#順に単語、品詞、出現頻度の情報
dic = dic.sort_values(by=['freq'], ascending=False)
dic = dic.reset_index(drop=True)#出現回数の降順にインデックスを付け直す
print(dic.head())
for i in range(3):#どんなふうになってるか確認
    print("type:",types[type_all[i]],"\n","name:",name_all[i],"\n","serif:",serif_all[i])

Using TensorFlow backend.


  words parts  freq
0     、    記号  8202
1     …    記号  5063
2     ！    記号  4801
3     の    助詞  4087
4     て    助詞  3968
type: cute 
 name: 島村卯月 
 serif: はじめまして、○○プロデューサーさん！ 島村卯月、17歳です。私、精一杯頑張りますから、一緒に夢叶えましょうね♪よろしくお願いしますっ！
type: cute 
 name: 島村卯月+ 
 serif: 〇〇プロデューサーさん、島村卯月です！もう覚えてもらえました？ 私、もっともっと頑張りますから、一緒に夢叶えましょうね♪
type: cute 
 name: 中野有香 
 serif: 押忍！……じゃなかった！お疲れ様ですっ！プロデューサーと言えば師匠も同然！！あたし、きっと最強のアイドルになりますから、ご指導ご鞭撻のほど、よろしくお願いします！！


次に辞書dicのidを利用してコーパスを作成する。このときセリフのつながりなどは全く考慮せずコーパスを作成していることに注意。一応タイプごとには分かれているので、タイプごとの言葉の傾向はとらえているかも?でもあまり期待しないほうが良い。  コーパスを作成する際は単語idのみを利用しているので数字の列にしか見えないことに注意。  
ついでにコーパスに現れる単語idから単語への変換を行う辞書id_to_wordと逆の変換を行うword_to_idを作成する。  

In [2]:
corpus = []
for i,serif in enumerate(serif_all):
    print(i,end = "\r")
    tokens = t.tokenize(serif)
    n = len(tokens)
    for j in range(n):
        dic_temp = dic[(dic['words'] == tokens[j].surface) & (dic['parts'] == tokens[j].part_of_speech[:2])]
        corpus.append(dic_temp.index[0])
corpus = np.array(corpus)
print(corpus)
print(len(corpus))
word_to_id = {}
id_to_word = {}
for i in range(len(dic)):
    word_to_id[dic.iloc[i].words] = i
    id_to_word[i] = dic.iloc[i].words

[882   0   8 ...  26  19   2]
146256


CBOWを利用するためのモジュールをインポートする。ついでにハイパラも設定。

In [3]:
from common import config
from common.trainer import Trainer
from common.optimizer import Adam
from common.cbow import CBOW
from common.util import create_contexts_target,to_cpu,to_gpu
#ハイパーパラメータの設定
window_size = 5#コンテキストの範囲
hidden_size = 100#中間層のサイズ、中間層は1つのみ用いる
batch_size = 100
max_epoch = 15

CBOWを利用して単語の分散表現word_vecsを得る。

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
vocab_size  = len(dic)
print(vocab_size)
contexts,target = create_contexts_target(corpus,window_size)#コーパスからコンテキストとターゲットを生成、これをもとに学習
model = CBOW(vocab_size, hidden_size, window_size, corpus)#今回は単純なCBOWを利用
optimizer = Adam()
trainer = Trainer(model,optimizer)
pkl_file = "cbow_params.pkl"

#既存のファイルから情報を取り出すにはこちらを利用する。
with open(pkl_file,"rb") as f:
    params = pickle.load(f)
    word_vecs = params["word_vecs"]
"""
#学習をやり直すときにはこちらを利用する必要がある。
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()
word_vecs = model.word_vecs
params = {}
params["word_vecs"] = word_vecs.astype(np.float16)
pkl_file = "cbow_params.pkl"
with open(pkl_file,"wb") as f:
    pickle.dump(params,f,-1)
"""

10157


'\n#学習をやり直すときにはこちらを利用する必要がある。\ntrainer.fit(contexts, target, max_epoch, batch_size)\ntrainer.plot()\nword_vecs = model.word_vecs\nparams = {}\nparams["word_vecs"] = word_vecs.astype(np.float16)\npkl_file = "cbow_params.pkl"\nwith open(pkl_file,"wb") as f:\n    pickle.dump(params,f,-1)\n'

単語の分散表現を利用して文章(セリフ)も分散表現にする。今回は簡単のため、文章を構成する単語の分散表現の相加平均を取ることとした。

In [5]:
serif_vec = []
i = 0
for serif in serif_all:#すべてのセリフを分散表現serif_vecに変換する。
    print(i,end = "\r")
    i += 1
    vec_temp = np.zeros(hidden_size)
    serif = serif.replace("\n","")
    tokens = t.tokenize(serif)
    for token in tokens:
        id_temp = word_to_id[token.surface]#単語の文字列からidへ
        vec_temp += word_vecs[id_temp]#idから分散表現へ
    vec_temp = vec_temp/len(tokens)#単語数で平均をとる
    serif_vec.append(vec_temp)
serif_vec = np.array(serif_vec)
print(serif_vec)

[[ 0.35981242 -0.26090834  0.3715744  ...  0.14067854  0.20926793
   0.6811693 ]
 [ 0.20117557 -0.17080596  0.37936771 ...  0.07709448  0.3895948
   0.76394098]
 [ 0.42473611 -0.06763916  0.40240462 ...  0.35961982  0.21317079
   0.5450548 ]
 ...
 [ 0.14392277  0.07146209  0.21087798 ... -0.01172624  0.22015928
   0.28453561]
 [ 0.23584797  0.29684316  0.32425789 ...  0.0282878   0.3372413
   0.26634498]
 [ 0.13959476  0.28073393  0.35515921 ...  0.31884275  0.0586932
   0.24050576]]


ランダムフォレストによるクラスタリング。入力データは文章の分散表現serif_vec,ターゲットはタイプの正解ラベルtype_allである。

In [6]:
import sklearn.ensemble
idx = np.arange(len(serif_vec))#どのように間違えたかを確認するためにこれが必要になる
idx_train,idx_test = sklearn.model_selection.train_test_split(np.arange(len(serif_vec)),test_size = 0.20)
train_x = serif_vec[idx_train]
train_y = np.array(type_all)[idx_train]
test_x = serif_vec[idx_test]
test_y = np.array(type_all)[idx_test]
model = sklearn.ensemble.RandomForestClassifier()
model.fit(train_x, train_y)
accuracy = model.score(train_x,train_y)
print("train:accuracy {0:.2%}".format(accuracy))
accuracy = model.score(test_x, test_y)
print('accuracy {0:.2%}'.format(accuracy))

C:\Users\yota-\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


train:accuracy 98.82%
accuracy 51.57%


サポートベクタマシン(SVM)によるクラスタリング。ランダムフォレストと同じ入力を与える。なぜかわからないがSVMの方が有意に精度はよさそう。

In [7]:
from sklearn.svm import SVC
model = SVC(kernel = "linear")
model.fit(train_x,train_y)
accuracy = model.score(train_x,train_y)
print("train:accuracy {0:.2%}".format(accuracy))
accuracy = model.score(test_x,test_y)
print('test:accuracy {0:.2%}'.format(accuracy))

train:accuracy 60.61%
test:accuracy 57.52%


どのようなときに間違えているのかを確認する。

In [9]:
type_pred = model.predict(test_x)#testデータに対してprediction
idx_error = idx_test[type_pred != np.array(type_all)[idx_test]]#testデータのうち間違えていたもののindexを抽出
print(len(type_pred),len(idx_error))
type_pred_all = model.predict(serif_vec)#全体のデータに対してprediction(このあとのために仕方なく使っている)
for i,idx_temp in enumerate(idx_error):
    if i > 4:
        break
    print("true:",types[type_all[idx_temp]],"pred:",types[type_pred_all[idx_temp]],"\n","name:",name_all[idx_temp],"\n","serif:",serif_all[idx_temp])

572 243
true: passion pred: cool 
 name: ［学園の魔女］メアリー・コクラン 
 serif: べ、別に怖くないワ！ただの下見だし、何かあっても撮影用の小道具って知ってるモン！それにココが七不思議の現場っていうのは、設定でしょ！…でも、もし何かが出たら…守ってよネ、ダーリン？
true: cool pred: passion 
 name: ［ロワイヤルスタイルNP］結城晴+ 
 serif: おっ、いいじゃん、これ！ やっとオレに似合う、まともな衣装がきたっつーか。こういうので踊って歌えっていうなら、やってやってもいいぜ！ ○○、オレのカッコイイところ、見てろよな！
true: cute pred: passion 
 name: ［ホームメイドハッピー］五十嵐響子+ 
 serif: みんなっ！今日は、おしとやかな私、卒業ですっ！ハジけるときは、ちゃんとハジけなきゃ♪ファンのみんなも、今日はおもいっきりハジけてくださいねっ！あっ、でも隣の人と仲良く！楽しんで！
true: passion pred: cool 
 name: ［海風の使者］浜口あやめ+ 
 serif: 忍びとして自然を味方につけることは心得ております。そう、つまり大海原であろうとあやめに隙はありません！伊賀の里での修行の成果を海軍の提督になりきって披露いたしましょう！
true: cute pred: passion 
 name: ［天下御免☆ガール］丹羽仁美 
 serif: やぁやぁ我こそは丹羽仁美！特売うなぎの蒲焼、勝ち取ったりぃ～肉厚で美味しそうでしょ、○○プロデューサー☆これで故郷の名物、ヒツマブシ作ったげるから、楽しみにしててよ！


混同行列を計算する。行が本当のタイプを表しており、列が予測されたタイプを表している。cuteの分類は難しいらしい。passionに分類されがち。半面、coolやpassionの分類は精度が高く、特にcoolとpassionを混同することは少ない。

In [10]:
confusion = np.zeros([3,3])
for i,idx_temp in enumerate(idx_test):
    confusion[type_all[idx_temp]][type_pred[i]] += 1
print(confusion)#混同行列の計算
for i in range(len(confusion)):#
    confusion[i] = confusion[i]/np.sum(confusion[i])
print(confusion)

[[ 87.  46.  69.]
 [ 39. 110.  27.]
 [ 33.  29. 132.]]
[[0.43069307 0.22772277 0.34158416]
 [0.22159091 0.625      0.15340909]
 [0.17010309 0.14948454 0.68041237]]
